In [1]:
import os
import pandas as pd
import json

In [2]:
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

c:\Users\kimso\anaconda3\envs\test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 데이터 불러오기

#### 1. 가사

In [3]:
lyrics1 = pd.read_csv("./dataset/label_result_song_short.csv")
lyrics2 = pd.read_csv("./dataset/translated_lyrics.csv")

lyrics = lyrics2.merge(lyrics1, on=["index", "id", "title", "singer", "genres", "lyrics"])

In [4]:
lyrics.head()

,index,id,title,singer,genres,lyrics,translated_lyrics,sentiment,sentiment_label
0,1,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Ay ay ay ay ay,Ay ay ay ay ay,2,분노
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,거울 속에 내 표정 봐 봐,거울 속에 내 표정 봐 봐,5,당황
2,3,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,느낌 So good 기다려온 D-day,느낌 그래서 좋아요. 기다려온 D-하루,0,기쁨
3,4,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,연습했던 손든 인사도 그대로 하면 돼,연습했던 손든 인사도 그대로 하면 돼,0,기쁨
4,5,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Hairstyle check하고 한 번 Turn around,머리스타일 확인하고 한 번 회전 주변,2,분노


#### 2. 소설

In [5]:
novel_final = pd.read_csv('./dataset/classified_novel1.csv')
data = novel_final.copy()

In [6]:
# data = data.sample(n=50000, random_state=42).reset_index(drop=True)

len(data)

885014

In [7]:
data.head()

,Column1,file_id,title,paragraph_id,text,labels,sentiment
0,0,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.1,01범보다 무서운 곶감,4.0,불안
1,1,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.2,화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴없이 꺼낸 첫 이야기이다. 화자로...,4.0,불안
2,2,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.3,그링깨. 사람이 어거지루는 못 살구. 응? 어거지루 안 되능 거여. 사람이 그링깨 ...,2.0,분노
3,3,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.4,"그래 옛날, 그 꼭감이라능 게 말여. 사람이 먹잖야 이케? 먹지마는. 그게 참 무성...",4.0,불안
4,4,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.5,"애기가 울어. 옛날에. 그래 할머니가 달갸(달래). 그때 호랭이가, 응? 그 집 문...",1.0,슬픔


### Preprocessing

In [8]:
print("NaN 개수:", data['text'].isna().sum())  # NaN 개수 확인
print("None 포함 여부:", data['text'].isnull().sum())  # None 개수 확인

NaN 개수: 79
None 포함 여부: 79


In [9]:
data['text'] = data['text'].fillna("")

In [10]:
print("NaN 개수:", data['text'].isna().sum())  # NaN 개수 확인
print("None 포함 여부:", data['text'].isnull().sum())  # None 개수 확인

NaN 개수: 0
None 포함 여부: 0


In [11]:
# 같은 title을 가진 데이터에서 100개씩 묶어 그룹화
data["group"] = data.groupby("title").cumcount() // 100  # 100개 단위 그룹 생성

In [12]:
data = data.groupby(["file_id", "group"]).agg({
    "title": "first",  # 대표 file_id
    "text": list,  # 100개씩 묶음
}).reset_index()

# 불필요한 sentiment 칼럼 삭제
data = data.drop(columns=["group", "file_id"])

data.head()

,title,text
0,이야기꾼 구연설화,"[01범보다 무서운 곶감, 화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴없이 ..."
1,이야기꾼 구연설화,"[<봄꿩은 제 울음에 저 죽는다>, 그 말과 같아서 사램이 잘못 되머넌 하는 말여...."
2,이야기꾼 구연설화,"[그래 그 여자가 쪄서 쌀얼 쪄서 밥얼 했어., “잡수라.”구., 그래 한 때를 몽..."
3,이야기꾼 구연설화,"[그랴. 그래 우리 인제 사춘 찾어간다는 얘기를 족~ 갈쳐중개,, “그러시냐구. 그..."
4,이야기꾼 구연설화,"[나와서 인제 그 집 먼지 인저 그, 그, 뭐여 면사무소 있는 디 나와서 인제, 거..."


In [13]:
data['text'] = data['text'].apply(lambda x: ' '.join(map(str, x)) if isinstance(x, list) else str(x))

# 불필요한 문자 제거
data['text'] = data['text'].str.replace(r"[\[\],']", "", regex=True)

In [14]:
data.head()

,title,text
0,이야기꾼 구연설화,01범보다 무서운 곶감 화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴없이 꺼낸...
1,이야기꾼 구연설화,<봄꿩은 제 울음에 저 죽는다> 그 말과 같아서 사램이 잘못 되머넌 하는 말여. 지...
2,이야기꾼 구연설화,그래 그 여자가 쪄서 쌀얼 쪄서 밥얼 했어. “잡수라.”구. 그래 한 때를 몽땅 먹...
3,이야기꾼 구연설화,그랴. 그래 우리 인제 사춘 찾어간다는 얘기를 족~ 갈쳐중개 “그러시냐구. 그러먼-...
4,이야기꾼 구연설화,나와서 인제 그 집 먼지 인저 그 그 뭐여 면사무소 있는 디 나와서 인제 거 와서 ...


In [15]:
data.loc[0]["text"]

'01범보다 무서운 곶감 화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴없이 꺼낸 첫 이야기이다. 화자로서 가장 쉽게 기억해낸 이야기인 셈이다. 설화 앞뒤에 교훈적 해석을 덧붙이고 있음은 화자의 습관화된 태도의 한 모습이기도 하다. 어려서 조모로부터 들었다고 했다. 그링깨. 사람이 어거지루는 못 살구. 응? 어거지루 안 되능 거여. 사람이 그링깨 뭐이냐 하먼 자연~간 제절루 되야지 어거지루는 못 살어 사람이. 그래 옛날 그 꼭감이라능 게 말여. 사람이 먹잖야 이케? 먹지마는. 그게 참 무성(무서운) 거여. 애기가 울어. 옛날에. 그래 할머니가 달갸(달래). 그때 호랭이가 응? 그 집 문앜이 와 섰어 지금. 옛날이는 호랭이가 말두 하구 그랴. 그래 인제 그 할머니가 애기를 달램성 왼갖 소리를 다 햐. ‘호랭이 왔다’구 해두 울구우 ‘괭이 왔다’두 울구 왼갖 소리를 다 해두 울어. 그랑깨 꼭감을 “아나 꼭감.” “아가 꼭감.” 그랑깨 꺄. 안 울어. 호랭이 생각이? ‘야아 이거 내가 젤 무선 짐승인디 말여? 나보다 무서웅 게 익구나. 호랭이 꼭감이 꼭갬이 참 무석구나.’ 싶어서 인제… 근디 인제 그 집이 말여. 왱깐이(외양간에) 가 숨었어. 인제. 무서서. 소도둑놈이 소를 몰러 와서 봉깨 함 마리가 익거덩? 가서 인제 소 손지 알구 말여. 꼬랭이를 집어탔어. 그러닝깨 이 호랭이는 꼭갬인지 알구 말여. 그 사람을. ‘아따! 꼭갬이로구나’ 싶어서 인제 그 눔을 실쿠 사알살 가 지금. 그래 도독놈은 인제 호랭인 줄 몰르구 말여 손주(소인 줄) 알구 말여 이 눔을 인제 갈기를 거머쥐구 집어타구서는 사알살 가는디. 아 이 뉨이 산 속으루 들어간다 인제? 그래 산 속으루 들어강깨 날이 샜어 인제. 그라자 곰이 말여? 곰을 만내서 곰이 하는 말이 “아 호새원(호생원)?” “왜 그라냐?” “아 왜 밥얼 실쿠 와요?” “예끼 놈! 이게 꼭감이다. 큰일 난다. 저리 가거라.” “아니 밥여요. 밥여. 그 참 맛있는 밥을 실쿠 가먼서 어째 그랍니까.” 아 그라다 가다 가닝깨 호랭이

In [16]:
# 데이터셋 클래스 정의
class NovelsDataset(Dataset):
    def __init__(self, data, tokenizer, text_column="input_text", max_length=128):
        self.data = data.reset_index(drop=True)  # 인덱스 리셋 (중복 방지)
        self.tokenizer = tokenizer
        self.text_column = text_column
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.loc[idx, self.text_column] # input_text 가져오기

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)

        # labels 생성 (input_ids 복사 및 패딩 토큰을 -100으로 설정)
        labels = input_ids.clone()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
        }

### Modeling

In [17]:
# KOGPT2 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

# 모델 초기화
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.bos_token_id = tokenizer.bos_token_id

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [18]:
dataset = NovelsDataset(data, tokenizer, text_column="text")
dataloader = DataLoader(dataset, batch_size=2)

In [19]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results/baseline',
    num_train_epochs=5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=5e-5,
    save_steps=200,
    save_total_limit=2,
    logging_steps=50,
    eval_strategy="epoch",
)

In [20]:
train_novels, eval_novels = train_test_split(data, test_size=0.1, random_state=42)

train_dataset = NovelsDataset(train_novels, tokenizer, text_column="text")
eval_dataset = NovelsDataset(eval_novels, tokenizer, text_column="text")

In [21]:
len(train_novels)

8074

In [22]:
# Trainer에 데이터셋 전달
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # 검증 데이터셋
)

In [23]:
sample = train_dataset[1]
print("Input IDs:", sample['input_ids'])
print("Attention Mask:", sample['attention_mask'])
print("Labels:", sample['labels'])

Input IDs: tensor([18519, 17582,  9117,  7652, 33298,  6841,  9316,  9135, 14668,   406,
         9018,  8198, 31187, 15648,  9673, 15218,  8702, 25830,  9078,  9338,
          388,   389, 17582,  9132,  7884,  6866, 46904,  7655,  7220,  9063,
        40182,  7478,  9201,  8771,  9894,  7991,   376, 10063,  8168,  7058,
        22882,   389,   739,  7753, 10690,  9109, 47711, 10063,  7967,   739,
         7753,  8210,  8006,   389, 17582, 46651,  9384, 10628,  7080, 47711,
          739, 29045,  8801,  8801,   389,  9543,  8602, 17720,  9351, 21538,
        12109, 30152, 12814,  9143, 10785,  9138,   389, 42411, 42506, 12583,
         9668, 10428,  6866, 13433, 22044, 12503,  8006,  6826, 27430,  9848,
        11001, 18739,  8406,  9016, 10428,  6866, 11577, 13176,  8705,  9068,
          739,  9168, 12503,  8263,  7969,  6889, 10027,  7285,  7210,  9395,
         8033,  9489, 17748, 24508, 43968,  9094, 15309, 15073,  9133,  7182,
         7183,   388, 13602,  7071, 50223, 10063,  80

In [24]:
# 모델 학습
trainer.train()

c:\Users\kimso\anaconda3\envs\test\lib\site-packages\transformers\integrations\sdpa_attention.py:53: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
0,4.311700,4.197879
1,3.979100,4.164830
2,3.692900,4.185980
3,3.454200,4.200779
4,3.264700,4.221184


TrainOutput(global_step=2520, training_loss=3.7454229385133773, metrics={'train_runtime': 1062.5728, 'train_samples_per_second': 37.993, 'train_steps_per_second': 2.372, 'total_flos': 2634868850688000.0, 'train_loss': 3.7454229385133773, 'epoch': 4.998761456527124})

In [25]:
device = torch.device("cuda")

In [26]:

# 이야기 생성 함수
def generate_story(lyrics_input):
    # 입력 토큰화
    input_ids = tokenizer.encode(lyrics_input, return_tensors='pt').to(device)

    # 출력 생성
    output = model.generate(
        input_ids, 
        max_new_tokens=150, # 새로 생성할 토큰의 개수를 제한
        num_return_sequences=1, 
        temperature=0.8, 
        top_k=50, 
        top_p=0.9, 
        repetition_penalty=1.2, 
        do_sample=True  # 샘플링 활성화
    )

    # 입력 길이 추적
    input_length = input_ids.shape[1]

    # 생성된 토큰 중 입력 토큰 이후의 부분만 디코딩
    generated_tokens = output[0][input_length:]
    generated_story = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return generated_story

In [27]:
lyrics1 = pd.read_csv("./dataset/label_result_song_short.csv")
lyrics2 = pd.read_csv("./dataset/translated_lyrics.csv")

lyrics = lyrics2.merge(lyrics1, on=["index", "id", "title", "singer", "genres", "lyrics"])

In [28]:
# groupby와 agg를 활용해 변환
lyrics_input = lyrics.groupby("id").agg({
    "title": "first",   # 첫 번째 값 유지
    "singer": "first",  # 첫 번째 값 유지
    "genres": "first",  # 첫 번째 값 유지
    "lyrics": list,  # 첫 번째 값 유지
    "translated_lyrics": list,  # 리스트로 묶음
}).reset_index()

In [29]:
lyrics_input.head()

,id,title,singer,genres,lyrics,translated_lyrics
0,418168,희재,성시경,"발라드, 국내영화","[햇살은 우릴 위해 내리고 , 바람도 서롤 감싸게 했죠 , 우리 웃음속에, 계절은 ...","[햇살은 우릴 위해 내리고 , 바람도 서롤 감싸게 했죠 , 우리 웃음속에, 계절은 ..."
1,418598,친구라도 될 걸 그랬어,거미 (GUMMY),R&B/Soul,"[벌써 넌 내가 편하니, 웃으며 인사 할 만큼, 까맣게 나를 잊었니, 네 곁에 있는...","[벌써 넌 내가 편하니, 웃으며 인사 할 만큼, 까맣게 나를 잊었니, 네 곁에 있는..."
2,711626,살다가,SG 워너비,발라드,"[살아도 사는 게 아니래, 너 없는 하늘에, 창 없는 감옥 같아서, 웃어도 웃는 게...","[살아도 사는 게 아니래, 너 없는 하늘에, 창 없는 감옥 같아서, 웃어도 웃는 게..."
3,1500196,내사람,SG 워너비,R&B/Soul,"[내 가슴속에 사는 사람 내가 그토록 아끼는 사람 , 너무 소중해 마음껏 안아보지도...","[내 가슴속에 사는 사람 내가 그토록 아끼는 사람 , 너무 소중해 마음껏 안아보지도..."
4,1854856,라라라,SG 워너비,발라드,"[그대는 참 아름다워요, 밤 하늘의 별빛보다 빛나요, 지친 나의 마음을 따뜻하게 감...","[그대는 참 아름다워요, 밤 하늘의 별빛보다 빛나요, 지친 나의 마음을 따뜻하게 감..."


In [53]:
i = 726
print(lyrics_input["title"][i])

홀씨


In [54]:
prompt = " ".join(lyrics_input["translated_lyrics"][i])

print(prompt)

내가 누울 자린 아마도 한참 더 위로 아니 적당히 미끈한 곳에 뿌리내리긴 싫어 내 뒤로 착착 따라붙어 다 예쁘게 줄지어 난 기어코 하늘에 필래 음, 뭐죠? 아 아주 작습니다. le아der 아슬아슬히 나는 홀씨 하나 또 다른 길을 향해서 비행 높은 옆으로 꽃 피우기 혹시 나의 안부를 묻는 누군가 있거든 전해줘 걔는 홀씨가 됐다구 날 따라, 언젠가 밖으로 나가 옆으로 승리 날 따라, 날아가 꼭대기루 여러분 말하자면 '후‘ 저는 어쩌면 파리 여러분 말하자면 '후‘ 그러면 저는 파리 날 따라, 심지어 무언가를 하지 않고 날개 날 따라, 떠올라 공중으루 여러분 말하자면 '후‘ 저는 어쩌면 파리 여러분 말하자면 '후' 그러면 저는 파리 다 날 볼 수 있게 날아 줄게 한가운데 시력을 위해 꼭 지참해 니 햇빛 올려보면 눈부셔 고소공포 하나도 안 무셔 따가운 태양과 무지 가까운 거리 까지 올라가 난 무심히 내려보리 구름을 골라타 간만에 한바탕 싹 어질러볼까 빙글빙그르 나는 홀씨 하나 가파른 바람을 타고 비행 높은 옆으로 꽃 피우기 혹시 나의 안부를 묻는 누군가 있거든 전해줘 걔는 홀씨가 됐다구 날 따라, 언젠가 밖으로 나가 옆으로 승리 날 따라, 날아가 꼭대기루 여러분 말하자면 '후‘ 저는 어쩌면 파리 여러분 말하자면 '후‘ 그러면 저는 파리 날 따라, 심지어 무언가를 하지 않고 날개 날 따라, 떠올라 공중으루 여러분 말하자면 '후‘ 저는 어쩌면 파리 여러분 말하자면 '후' 그러면 저는 파리 앞길이 만만치 않아도 엄살은 뒤로 내 선택이야 늘 그랬듯이 쉬울 확률은 0 남은 거 탈탈 털어줄게 모두 행운을 빌어 구태여 인사하고 갈래 5월 신 무언가를 해야 합니다. 바이러스 야 보세요. 야


In [55]:
# 가사로 이야기 생성
generated_story = generate_story(prompt)
print("생성된 이야기:", generated_story)

생성된 이야기: 옹~!」 그러니까....... 그럼 그렇게 말입니다. 이런 이야기 그만 들리는 순간 갑자기 저도 저는 나 역시 그런 상황이 되고 마냥 제발 길인 듯하네...... 하고 멈출 것 같은 이겠군요─ ....... "저... 그거야말이에요." ....... "그러면 네가 어디로 가시는 건데요?" 그건....... 맞습니다...... 당신은 지금 막 도착하기 전에 제가 할 말이 있는 겁니다 이제 곧장.... 그래......." "음...... 그렇습니다 그렇지 않으면......? -_-...... 그... 아...." 잠시 후에...... "예 알았어요?" 정말로 두 분께선 그냥 대답을 하시려다가 「얘들 좀 봐


In [49]:
# 가사로 이야기 생성
# lyrics_input = ""
# generated_story = generate_story(lyrics_input, tag1="2", tag2="2")
# print("생성된 이야기:", generated_story)